# 사전 작업

## 모듈 로드

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import seaborn as sns
import gc
from tqdm import tqdm_notebook

In [2]:
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import log_loss
from sklearn.model_selection import KFold, StratifiedKFold

In [3]:
import warnings
warnings.filterwarnings('ignore')

In [4]:
pd.set_option('display.max_columns', 400)

# 통합 데이터

In [5]:
def drop_features(data, keyword, debug=False):
    if debug:
        data = data[:1000]
        
    logits = []
    for col in data.columns:
        if keyword in col:
            logits.append(col)
        else:
            pass
    
    return logits

## 데이터 로드

In [6]:
path = './data/'

In [48]:
train = pd.read_csv(path + 'train_v4.csv')
test = pd.read_csv(path + 'test_v4.csv')

In [49]:
train.new_card_id_size.fillna(0, inplace=True)
test.new_card_id_size.fillna(0, inplace=True)

In [9]:
raw_history = pd.read_csv(path + 'historical_transactions.csv')

In [10]:
history = raw_history.copy()

## Feature Engineering

In [50]:
def null_cnt(x):
    return np.sum(x.isna() * 1)

In [51]:
def null_rate(x):
    return np.sum(x.isna() * 1) / len(x)

In [52]:
from scipy import stats
def mode(x):
    return stats.mode(x)[0][0]

In [53]:
def most_value_cnt(x):
    return x.value_counts().values[0]

In [41]:
history.head(3)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37


### 거래가 승인 되고, 오프라인 거래의 city_id

In [54]:
temp = history[history.authorized_flag == 'Y']
temp = temp[temp.category_1 == 'N']

In [55]:
temp = temp.groupby('card_id').agg({'city_id': [mode, 'nunique']})
temp.head(1)

city_id        
                   mode nunique
card_id                        
C_ID_00007093c1     244       3

In [56]:
temp.columns = ['hist_city_id_mode_authorized_flag_category_1_Y', 'hist_city_id_nunique_authorized_flag_category_1_Y']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_city_id_mode_authorized_flag_category_1_Y,hist_city_id_nunique_authorized_flag_category_1_Y
0,C_ID_00007093c1,244,3


In [57]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [58]:
del temp
gc.collect()

43

### 거래가 승인된 category_1

In [78]:
temp = history[history.authorized_flag == 'Y']

In [79]:
temp.category_1 = temp.category_1.map({'Y':1, 'N':0})

In [80]:
temp = temp.groupby('card_id').agg({'category_1':['mean', 'sum', 'size']})
temp.head(1)

category_1         
                      mean sum size
card_id                            
C_ID_00007093c1   0.210526  24  114

In [81]:
temp.columns = ['hist_category_1_authorized_flag_Y_mean', 'hist_category_1_authorized_flag_Y_sum', 'hist_category_1_authorized_flag_Y_size']
temp.reset_index(inplace=True)
temp.head(1)

,card_id,hist_category_1_authorized_flag_Y_mean,hist_category_1_authorized_flag_Y_sum,hist_category_1_authorized_flag_Y_size
0,C_ID_00007093c1,0.210526,24,114


In [82]:
temp.hist_category_1_authorized_flag_Y_mean = np.round(temp.hist_category_1_authorized_flag_Y_mean, 4)

In [83]:
train = train.merge(temp, on='card_id', how='left')
test = test.merge(temp, on='card_id', how='left')

In [84]:
del temp
gc.collect()

21

## Feature Selection

In [86]:
for df in [train, test]:
    df['temp'] = 1 - df.hist_category_1_label_mean
    df['hist_category_1_authorized_flag_Y_0_cnt'] = df.hist_category_1_authorized_flag_Y_size - df.hist_category_1_authorized_flag_Y_sum

In [87]:
train.head(3)

,card_id,feature_1,feature_2,feature_3,first_active_month,outliers,target,first_active,first_active_year,first_active_elapsed_time_from_trade,first_active_total_day,hist_authorized_flag_label_mode,hist_authorized_flag_label_sum,hist_authorized_flag_label_mean,hist_card_id_size,hist_city_id_mode,hist_city_id_nunique,hist_category_1_label_mode,hist_category_1_label_sum,hist_category_1_label_mean,hist_installments_mode,hist_installments_sum,hist_installments_mean,hist_installments_var,hist_installments_max,hist_installments_min,hist_installments_null_cnt,hist_category_3_label_mode,hist_category_3_label_mean,hist_merchant_category_id_mode,hist_merchant_category_id_nunique,hist_merchant_id_nunique,hist_month_lag_mode,hist_month_lag_sum,hist_month_lag_mean,hist_month_lag_var,hist_month_lag_max,hist_month_lag_min,hist_month_lag_skew,hist_purchase_date_max,hist_purchase_date_min,hist_purchase_amount_sum,hist_purchase_amount_mean,hist_purchase_amount_var,hist_purchase_amount_max,hist_purchase_amount_min,hist_purchase_amount_skew,hist_purchase_amount_over_550,hist_category_2_mode,hist_category_2_mean,hist_state_id_mode,hist_state_id_nunique,hist_subsector_id_mode,hist_subsector_id_nunique,hist_purchase_amount_trim_sum,hist_purchase_amount_trim_mean,hist_purchase_amount_trim_var,hist_purchase_amount_trim_max,hist_purchase_amount_trim_min,hist_purchase_amount_trim_skew,hist_purchase_year_mode,hist_purchase_year_nunique,hist_purchase_year_mean,hist_purchase_year_max,hist_purchase_year_min,hist_purchase_month_mode,hist_purchase_month_nunique,hist_purchase_month_mean,hist_purchase_month_max,hist_purchase_month_min,hist_purchase_day_mode,hist_purchase_day_nunique,hist_purchase_day_mean,hist_purchase_day_var,hist_purchase_day_max,hist_purchase_day_min,hist_purchase_day_skew,hist_purchase_hour_mode,hist_purchase_hour_nunique,hist_purchase_hour_mean,hist_purchase_hour_var,hist_purchase_hour_max,hist_purchase_hour_min,hist_purchase_hour_skew,hist_purchase_dayofweek_mode,hist_purchase_dayofweek_nunique,hist_purchase_dayofweek_mean,hist_purchase_dayofweek_max,hist_purchase_dayofweek_min,hist_purchase_weekofyear_mode,hist_purchase_weekofyear_nunique,hist_purchase_weekofyear_mean,hist_purchase_weekofyear_max,hist_purchase_weekofyear_min,hist_purchase_weekend_mode,hist_purchase_weekend_sum,hist_purchase_weekend_mean,hist_price_sum,hist_price_mean,hist_price_max,hist_price_min,hist_price_var,hist_month_diff_max,hist_month_diff_min,hist_month_diff_mean,hist_month_diff_var,hist_month_diff_skew,hist_Christmas_Day_2017_mean,hist_Mothers_Day_2017_mean,hist_fathers_day_2017_mean,hist_Children_day_2017_mean,hist_Valentine_Day_2017_mean,hist_Black_Friday_2017_mean,hist_Mothers_Day_2018_mean,hist_duration_mean,hist_duration_min,hist_duration_max,hist_duration_var,hist_duration_skew,hist_amount_month_ratio_mean,hist_amount_month_ratio_min,hist_amount_month_ratio_max,hist_amount_month_ratio_var,hist_amount_month_ratio_skew,hist_category_2_mean_mean,hist_category_2_min_mean,hist_category_2_max_mean,hist_category_2_sum_mean,hist_category_3_mean_mean,hist_category_3_min_mean,hist_category_3_max_mean,hist_category_3_sum_mean,hist_purchase_date_diff,hist_purchase_date_average,hist_purchase_date_uptonow,hist_purchase_date_uptomin,new_authorized_flag_mode,new_authorized_flag_sum,new_authorized_flag_mean,new_card_id_size,new_city_id_mode,new_city_id_nunique,new_category_1_mode,new_category_1_sum,new_category_1_mean,new_installments_mode,new_installments_sum,new_installments_mean,new_installments_var,new_installments_max,new_installments_min,new_installments_null_cnt,new_category_3_mode,new_category_3_mean,new_merchant_category_id_mode,new_merchant_category_id_nunique,new_merchant_id_nunique,new_month_lag_mode,new_month_lag_sum,new_month_lag_mean,new_month_lag_var,new_month_lag_max,new_month_lag_min,new_month_lag_skew,new_purchase_date_max,new_purchase_date_min,new_purchase_amount_sum,new_purchase_amount_mean,new_purchase_amount_var,new_purchase_amount_max,new_purchase_am

In [139]:
trainable_feature = [
    'feature_1', 'feature_2', 'feature_3', 
    'first_active_month', 'first_active_year', 'first_active_elapsed_time_from_trade', 'first_active_total_day',
    'hist_card_id_size', 'new_card_id_size',
    'hist_authorized_flag_label_mean', 'hist_authorized_flag_label_sum',
    'hist_city_id_mode_authorized_flag_category_1_Y', 'hist_city_id_nunique_authorized_flag_category_1_Y',
    'hist_category_1_mean']

In [140]:
param = {'num_leaves': 31,
         'min_data_in_leaf': 30, 
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 1,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 8,
         "random_state": 4590}

In [141]:
folds = StratifiedKFold(n_splits=5, shuffle=True, random_state=15)

# train_columns = [f for f in train.columns if f not in FEATS_EXCLUDED]
train_columns = trainable_feature

oof_lgb = np.zeros(len(train))
predictions_lgb = np.zeros(len(test))
feature_importance = pd.DataFrame()

for fold_, (trn_idx, val_idx) in enumerate(folds.split(train, train['outliers'].values)):    
    print("fold n°{}".format(fold_))
    trn_data = lgb.Dataset(train.iloc[trn_idx][train_columns], label=train.iloc[trn_idx]['target'])
    val_data = lgb.Dataset(train.iloc[val_idx][train_columns], label=train.iloc[val_idx]['target'])

    num_round = 10000
    clf = lgb.train(param, trn_data, num_round, valid_sets = [trn_data, val_data], verbose_eval=100, early_stopping_rounds = 200)
    oof_lgb[val_idx] = clf.predict(train.iloc[val_idx][train_columns], num_iteration=clf.best_iteration)
    
    predictions_lgb += clf.predict(test[train_columns], num_iteration=clf.best_iteration) / folds.n_splits
    
    fold_importance = pd.DataFrame()
    fold_importance["Feature"] = train_columns
    fold_importance["importance"] = clf.feature_importance()
    fold_importance["fold"] = fold_ + 1
    feature_importance = pd.concat([feature_importance, fold_importance], axis=0)
    
print("CV score: {:<8.5f}".format(mean_squared_error(train.target.values, oof_lgb)**0.5))

fold n°0
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.64886	valid_1's rmse: 1.6557
[200]	training's rmse: 1.63559	valid_1's rmse: 1.64533
[300]	training's rmse: 1.63059	valid_1's rmse: 1.64312
[400]	training's rmse: 1.62729	valid_1's rmse: 1.64248
[500]	training's rmse: 1.62458	valid_1's rmse: 1.64227
[600]	training's rmse: 1.62218	valid_1's rmse: 1.64223
[700]	training's rmse: 1.61992	valid_1's rmse: 1.64219
[800]	training's rmse: 1.61778	valid_1's rmse: 1.64226
[900]	training's rmse: 1.61575	valid_1's rmse: 1.64229
Early stopping, best iteration is:
[703]	training's rmse: 1.61985	valid_1's rmse: 1.64218
fold n°1
Training until validation scores don't improve for 200 rounds.
[100]	training's rmse: 1.64927	valid_1's rmse: 1.65425
[200]	training's rmse: 1.63614	valid_1's rmse: 1.64276
[300]	training's rmse: 1.63124	valid_1's rmse: 1.64018
[400]	training's rmse: 1.62806	valid_1's rmse: 1.63932
[500]	training's rmse: 1.62537	valid_1's rmse: 1.638

score : 3.80887

In [142]:
sub = pd.read_csv('./data/sample_submission.csv')

In [143]:
sub.target = predictions_lgb

In [145]:
sub.to_csv('./data/sub_tt.csv', index=False)